In [ ]:
!git clone https://github.com/AleksL04/ML-Project.git
%cd ML-Project

In [ ]:
!git pull

In [ ]:
%%capture
!pip install --upgrade gensim
!gdown --folder 1rl_TJMHtcP-S0fTHNUpr9oi8Ndz2Y5z4 -O project_data
!gdown --folder 1RmC_We2lfsJontwrxlc4dzqEY_mNy-UW -O word2vec_model

!pip install --upgrade scikeras
!pip install "scikit-learn<1.6"


In [ ]:
ls -l

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv('project_data/train.csv')
valid_df = pd.read_csv('project_data/valid.csv')
test_df = pd.read_csv('project_data/test.csv')

In [ ]:
from process_df import process_df

X_train = process_df(train_df)
y_train = train_df['label'].values

X_valid = process_df(valid_df)
y_valid = valid_df['label'].values

X_test = process_df(test_df)
y_test = test_df['label'].values

In [ ]:
from model_code.LSTM_code import trainLSTM, train_CV_LSTM
from model_code.CNN_code import trainCNN, train_CV_CNN
from model_code.Blender_code import trainBlender
from model_code.Bayes_code import trainBayes

In [ ]:
train_CV_LSTM(X_train, y_train, X_valid, y_valid)
#train_CV_CNN(X_train, y_train, X_valid, y_valid)

In [ ]:
LSTM_model = trainLSTM(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
CNN_model = trainCNN(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = CNN_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
Bayes_model = trainBayes(train_df, valid_df)

In [ ]:
val_pred_LSTM = LSTM_model.predict(X_valid)
val_pred_CNN = CNN_model.predict(X_valid)

X_meta_train = np.column_stack((val_pred_LSTM, val_pred_CNN))

y_meta_train = y_valid

In [ ]:
Blender_model = trainBlender(X_meta_train, y_meta_train)

In [ ]:
test_pred_LSTM = LSTM_model.predict(X_test)
test_pred_CNN = CNN_model.predict(X_test)

X_meta_test = np.column_stack((test_pred_LSTM, test_pred_CNN))

test_loss, test_acc = Blender_model.evaluate(X_meta_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")